Criação das Instâncias de serviços

In [1]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.transforms import Map, DropFields, ApplyMapping

sc = SparkContext()
context = GlueContext(sc)

In [2]:
path = "s3://network.cubo.datalake/airtable/raw/prod/"
path = path + "2023-01-17-21-08-44/data/"
path = path + "Selo Cubo Startups/Selo Cubo 2023/📝 Respostas Selo Cubo/"
dataframe = context.create_dynamic_frame.from_options(
                connection_type='s3',
                connection_options={
                    'paths': [path],
                    'recurse': True
                },
                format='json'
            )
dataframe.count()

s3://network.cubo.datalake/airtable/raw/prod/2023-01-17-21-08-44/data/Selo Cubo Startups/Selo Cubo 2023/📝 Respostas Selo Cubo/


343

In [ ]:
dataframe.printSchema()

In [3]:
mapping = []
for item in dataframe.unnest().toDF().dtypes:
    if item[0].split('.')[0] == "fields":
        if len(item[0].split(".")) == 2:
            mapping.append((item[0], item[0].split('.')[1]))
    else:
        mapping.append((item[0], item[0].split('.')[0]))

In [15]:
newDf = dataframe.apply_mapping(mapping).resolveChoice(specs=[
    ("Faturamento 2022", "cast:decimal"),
    ("Valor a captar / Faturamento previsto", "cast:decimal"),
    ("Previsão do faturamento 2023", "cast:decimal"),
    ("Captable", "cast:decimal"),
    ("Faturamento 2021", "cast:decimal"),
    ("Turnover", "cast:decimal"),
    ("Δ Fat 21-22", "cast:decimal"),
    ("Δ Clientes 21-22", "cast:decimal"),
    ("Δ Fat 22-23", "cast:decimal"),
    ("Δ Fat 21-23", "cast:decimal"),
    ("Δ Fat 21-22/22-23", "cast:decimal"),
    ("Δ Clientes 22-23", "cast:decimal"),
    ("Δ Clientes 21-23", "cast:decimal"),
    ("Δ Clientes 21-22/22-23", "cast:decimal"),
    ("Δ Time 22-23", "cast:decimal"),
    ("Δ Time 21-22/22-23", "cast:decimal"),
    ("PERFORMANCE", "cast:decimal"),
    ("CRESCIMENTO", "cast:decimal"),
    ("INVESTIMENTO", "cast:decimal"),
    ("PESSOAS", "cast:decimal"),
    ("Δ Time 21-23", "cast:decimal"),
    ("Δ Time 21-22", "cast:decimal"),
    ("Δ Time 21-22/22-23", "cast:decimal")
])
newDf.printSchema()


root
|-- id: string
|-- createdTime: string
|-- Data de fundação: string
|-- Descrição: string
|-- Impacto de Governança: array
|    |-- element: string
|-- Valor a ser captado: int
|-- Conexão com startups: array
|    |-- element: string
|-- Time 2021: int
|-- Segmento de Mercado: string
|-- Termo e política de privacidade: boolean
|-- Corporação top of mind: array
|    |-- element: string
|-- Desafios para 2023: array
|    |-- element: string
|-- Diversidade no Time: array
|    |-- element: string
|-- URL Pitch Deck: string
|-- Conexões Comunidade Cubo Itaú: array
|    |-- element: string
|-- Email Fundador 1: string
|-- Modelo de Receita: array
|    |-- element: string
|-- Outros fundos: string
|-- One Page Selo Cubo: array
|    |-- element: struct
|    |    |-- id: string
|    |    |-- url: string
|    |    |-- filename: string
|    |    |-- size: int
|    |    |-- type: string
|    |    |-- thumbnails: struct
|    |    |    |-- small: struct
|    |    |    |    |-- url: string
|  

In [18]:
possibilidades = []
def mapAll(item):
    for itemLoop in item["Desafios para 2023"]:
        if itemLoop not in possibilidades:
            possibilidades.append(itemLoop)

    return item

teste = newDf.select_fields(paths=["Impacto de Governança", "Desafios para 2023", "id", "Conexões Comunidade Cubo Itaú"])
teste.printSchema()

# possibilidades = []
# def mapAll(item):
#     for itemLoop in item["Desafios para 2023"]:
#         if itemLoop not in possibilidades:
#             possibilidades.append(itemLoop)

#     return item

# teste = newDf.toDF().select('Desafios para 2023').distinct().collect()

root
|-- Impacto de Governança: array
|    |-- element: string
|-- Desafios para 2023: array
|    |-- element: string
|-- id: string
|-- Conexões Comunidade Cubo Itaú: array
|    |-- element: string



In [19]:
teste.relationalize(
    "l_root", "s3://network.cubo.datalake/airtable/proc/prod/metadata"
)

In [ ]:
# path = "s3://network.cubo.datalake/airtable/proc/prod/"
# path = path + "2022-12-29-15-21-18/data/"
# path = path + "Selo Cubo Startups/Selo Cubo 2023/📝 Respostas Selo Cubo/"
# context.write_dynamic_frame.from_options(
#             frame=newDf,
#             connection_type="s3",
#             format="parquet",
#             connection_options={
#                 "path": path,
#                 "partitionKeys": [],
#             },
#             format_options={"compression": "gzip"},
#             transformation_ctx="S3Destination",
#         )